In [ ]:
!pip install -q pinecone openai PyMuPDF
!pip install -q sentence-transformers
!pip install -q pdfminer.six
!pip install -q transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 45.2 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF
import pinecone
from openai import OpenAI
from google.colab import files
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec

from pathlib import Path
from typing import List
from pdfminer.high_level import extract_text
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
# embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # dim=384
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")  # dim=384

# def get_embedding(text):
#     return embedding_model.encode(text).tolist()

def get_embedding(t):
  return embedding_model.encode(t).astype("float32").tolist()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
pc = Pinecone(api_key=PINECONE_API_KEY)

# Si el índice existe, lo borramos para recrearlo
if "cv-index" in pc.list_indexes().names():
    pc.delete_index("cv-index")

pc.create_index(
    name='cv-index',
    dimension=384,
    metric='cosine',
    spec=ServerlessSpec(cloud='aws', region='us-east-1')
)

index_info = pc.describe_index("cv-index")
index = pc.Index(host=index_info.host)

In [ ]:
import os

def extract_text_from_pdf(file_path: str) -> str:
    return extract_text(file_path)  # usa pdfminer.six

def create_document(text: str, metadata=None) -> Document:
    metadata = metadata or {}
    return Document(page_content=text, metadata=metadata)

def chunk_text(document: Document, chunk_size=500, chunk_overlap=100) -> List[Document]:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return splitter.split_documents([document])

# --- MULTI-PDF ---

def chunk_pdfs(
    pdf_paths: List[str],
    chunk_size: int = 1000,
    chunk_overlap: int = 200
) -> List[Document]:
    """Procesa múltiples PDFs y devuelve una lista plana de chunks (Documents)."""
    all_chunks: List[Document] = []
    for p in pdf_paths:
        text = extract_text_from_pdf(p) or ""
        if not text.strip():
            print(f"⚠️ Vacío o ilegible: {p}")
            continue
        doc = create_document(text, metadata={"source": str(p)})
        chunks = chunk_text(doc, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        # Agrego metadata útil a cada chunk
        for i, ch in enumerate(chunks):
            ch.metadata.update({"chunk_id": i, "namespace": os.path.splitext(os.path.basename(p))[0]})
        all_chunks.extend(chunks)
    return all_chunks

def chunk_pdfs_in_dir(
    folder: str,
    pattern: str = "*.pdf",
    **kwargs
) -> List[Document]:
    """Busca PDFs en una carpeta (recursivo opcional usando '**/*.pdf') y chunea todo."""
    files = [str(p) for p in Path(folder).glob(pattern)]
    return chunk_pdfs(files, **kwargs)

# --- USO ---

# Opción A: lista de archivos
# pdfs = ["cv1.pdf", "cv2.pdf", "cv3.pdf"]
# chunks = chunk_pdfs(pdfs, chunk_size=1000, chunk_overlap=200)

# Opción B: carpeta completa (no recursivo)
chunks = chunk_pdfs_in_dir("./cvs", pattern="*.pdf", chunk_size=200, chunk_overlap=50)

# Opción C: carpeta recursiva
# chunks = chunk_pdfs_in_dir("./cvs", pattern="**/*.pdf", chunk_size=1000, chunk_overlap=200)

print(f"Total de chunks: {len(chunks)}")
print(chunks[0].metadata, chunks[0].page_content[:120], "...")


Total de chunks: 28
{'source': 'cvs/pablo.pdf', 'chunk_id': 0, 'namespace': 'pablo'} PABLO ENRIQUE SISTO 

CABA, Argentina 
Tel: 11 3483 2001 
Email: pablosisto0@gmail.com 
Portfolio: https://pablosisto.gi ...


In [ ]:
chunks

[Document(metadata={'source': 'cvs/pablo.pdf', 'chunk_id': 0, 'namespace': 'pablo'}, page_content='PABLO ENRIQUE SISTO \n\nCABA, Argentina \nTel: 11 3483 2001 \nEmail: pablosisto0@gmail.com \nPortfolio: https://pablosisto.github.io/porfolio/'),
 Document(metadata={'source': 'cvs/pablo.pdf', 'chunk_id': 1, 'namespace': 'pablo'}, page_content='PERFIL PROFESIONAL \nDesarrollador Backend con experiencia en Java (Spring Boot) y .NET (C#), especializado en \nla creación y mantenimiento de APIs REST y la integración de servicios externos.'),
 Document(metadata={'source': 'cvs/pablo.pdf', 'chunk_id': 2, 'namespace': 'pablo'}, page_content='Conocimientos sólidos en bases de datos MySQL y SQL Server, despliegues en AWS y control \nde versiones con Git. Enfocado en la optimización de sistemas CRM y la implementación de'),
 Document(metadata={'source': 'cvs/pablo.pdf', 'chunk_id': 3, 'namespace': 'pablo'}, page_content='soluciones escalables y seguras. Apasionado por la mejora continua y el trabaj

In [ ]:
from collections import defaultdict

# Agrupamos los vectors por namespace
vectors_by_namespace = defaultdict(list)

for chunk in chunks:
    namespace = chunk.metadata.get("namespace", "default")
    vector = {
        "id": f"{Path(chunk.metadata['source']).stem}-{chunk.metadata['chunk_id']}",
        "values": get_embedding(chunk.page_content),
        "metadata": {
            **chunk.metadata,
            "text": chunk.page_content
        }
    }
    vectors_by_namespace[namespace].append(vector)


In [ ]:
for namespace, vectors in vectors_by_namespace.items():
    print(f"📤 Upserteando {len(vectors)} vectores en namespace: '{namespace}'")
    index.upsert(vectors=vectors, namespace=namespace)

📤 Upserteando 19 vectores en namespace: 'pablo'
📤 Upserteando 9 vectores en namespace: 'ramiro'


In [ ]:
# hago esto para asegurarme de que ya está
import time
while True:
    d = pc.describe_index("cv-index")
    if getattr(d, "status", {}).get("ready"):
        break
    time.sleep(1)

In [ ]:
pregunta = "What languages can the candidate speak?"
embedding_pregunta = get_embedding(pregunta)

res = index.query(vector=embedding_pregunta, top_k=5, include_metadata=True, namespace="ramiro")

matches = res.get('matches', [])
if not matches:
    print("⚠️ No se encontraron resultados")
else:
    print("CV más cercano:", matches[0]['id'])
    print("Texto:", matches[0]['metadata']['source'])

CV más cercano: ramiro-7
Texto: cvs/ramiro.pdf


In [ ]:
matches

[{'id': 'ramiro-7',
  'metadata': {'chunk_id': 7.0,
               'namespace': 'ramiro',
               'source': 'cvs/ramiro.pdf',
               'text': 'Escuela Superior de Comercio Carlos Pellegrini (ESCCP)\n'
                       'High School Degree with Accounting Specialization | '
                       '2008 - 2012\n'
                       '\n'
                       'Languages\n'
                       'Spanish\n'
                       'Native.\n'
                       '\n'
                       'English: \n'
                       'Proficient.\n'
                       '\n'
                       'Skills'},
  'score': 0.290181428,
  'values': []},
 {'id': 'ramiro-4',
  'metadata': {'chunk_id': 4.0,
               'namespace': 'ramiro',
               'source': 'cvs/ramiro.pdf',
               'text': 'Vanilla Javascript\n'
                       '\n'
                       'English-Spanish Translations\n'
                       'Main Studio | 2013 - 2015\n'
          